In [9]:
import csv
import ipaddress

valuesDict = {}
totalNum = 0
index = 0

file = open("B:/guillem/Downloads/ip2asn-v4.tsv", 'r')
for row in csv.reader(file, delimiter='\t'):
    if row[2] != '0':
        startIp = ipaddress.ip_address(unicode(row[0]))
        endIp = ipaddress.ip_address(unicode(row[1]))
        ipCount = int(endIp)-int(startIp)+1
        totalNum += ipCount
        if row[2] not in valuesDict:
            valuesDict[row[2]] = ipCount
        else:
            valuesDict[row[2]] += ipCount
    index += 1
    if index%100000 == 0:
        print(index)
file.close()
print('Total number of ipv4 ips: ' + str(totalNum))

100000
200000
300000
400000
500000
600000
700000
800000
900000
Total number of ipv4 ips: 2842786508


In [11]:
countryToRir = {}

file = open("B:/guillem/Downloads/convertcsv.csv", 'r')
for line in file:
    lineVec = line.split(';')
    countryToRir[lineVec[1]] = lineVec[3].split('\n')[0]
print(countryToRir)

{'BD': 'APNIC', 'BE': 'RIPE NCC', 'BF': 'AFRINIC', 'BG': 'RIPE NCC', 'BA': 'RIPE NCC', 'BB': 'ARIN', 'WF': 'APNIC', 'BL': 'ARIN', 'BM': 'ARIN', 'BN': 'APNIC', 'BO': 'LACNIC', 'BH': 'RIPE NCC', 'BI': 'AFRINIC', 'BJ': 'AFRINIC', 'BT': 'APNIC', 'JM': 'ARIN', 'BV': 'ARIN', 'BW': 'AFRINIC', 'WS': 'APNIC', 'BQ': 'LACNIC', 'BR': 'LACNIC', 'BS': 'ARIN', 'JE': 'RIPE NCC', 'BY': 'RIPE NCC', 'BZ': 'LACNIC', 'RU': 'RIPE NCC', 'RW': 'AFRINIC', 'RS': 'RIPE NCC', 'TL': 'APNIC', 'RE': 'AFRINIC', 'TM': 'RIPE NCC', 'TJ': 'RIPE NCC', 'RO': 'RIPE NCC', 'TK': 'APNIC', 'GW': 'AFRINIC', 'GU': 'APNIC', 'GT': 'LACNIC', 'GS': 'LACNIC', 'GR': 'RIPE NCC', 'GQ': 'AFRINIC', 'GP': 'ARIN', 'JP': 'APNIC', 'GY': 'LACNIC', 'GG': 'RIPE NCC', 'GF': 'LACNIC', 'GE': 'RIPE NCC', 'GD': 'ARIN', 'GB': 'RIPE NCC', 'GA': 'AFRINIC', 'GN': 'AFRINIC', 'GM': 'AFRINIC', 'GL': 'RIPE NCC', 'GI': 'RIPE NCC', 'GH': 'AFRINIC', 'OM': 'RIPE NCC', 'TN': 'AFRINIC', 'JO': 'RIPE NCC', 'HR': 'RIPE NCC', 'HT': 'LACNIC', 'HU': 'RIPE NCC', 'HK': 'AP

In [30]:
import json
from urllib2 import urlopen

ASN_Map = {}

numPagesPerOrg = 147;
for i in range(1, numPagesPerOrg+1):
    contents = json.loads(urlopen( "http://as-rank.caida.org/api/v1/asns?populate=1&page=" + str(i)).read());
    for item in contents["data"]:
        id = ''
        if 'id' in item:
            id = item['id']
        else:
            print('Error')
        country = ''
        if 'country' in item:
            country = item['country']
            if item['country'] == 'EU':
                if not 'country_name' in item or item['country_name'] != 'European Union':
                    print(item)
        if id != '' and country != '':
            ASN_Map[id] = country
    if i%20 == 0:
        print(str(i) + " out of " + str(numPagesPerOrg));
print('done!')

20 out of 147
40 out of 147
60 out of 147
80 out of 147
100 out of 147
120 out of 147
140 out of 147
done!


In [31]:
listEU = []
file = open("B:/guillem/Downloads/EU_countries.txt", 'r')
for line in file:
    listEU.append(line.split('-')[0].split()[0])
print(listEU)

['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PO', 'PT', 'RO', 'SE', 'SI', 'SK']


In [32]:
RirIPs = {}
total = 0
US = 0
CN = 0
UE = 0

for asn, ips in valuesDict.items():
    if asn in ASN_Map and (ASN_Map[asn] in countryToRir or ASN_Map[asn] == 'EU'):
        rir = 'RIPE NCC'
        if ASN_Map[asn] in countryToRir:
            rir = countryToRir[ASN_Map[asn]]
        elif ASN_Map[asn] != 'EU':
            print('Error: ' + str(ASN_Map[asn]))
        if ASN_Map[asn] == 'CN':
            CN += float(ips)
        elif ASN_Map[asn] == 'US':
            US += float(ips)
        elif ASN_Map[asn] == 'EU' or ASN_Map[asn] in listEU:
            UE += float(ips)
        if rir in RirIPs:
            RirIPs[rir] += float(ips)
        else:
            RirIPs[rir] = float(ips)
    else:
        if 'unknown' in RirIPs:
            RirIPs['unknown'] += float(ips)
        else:
            RirIPs['unknown'] = float(ips)
    total += float(ips)

for rir, ips in RirIPs.items():
    RirIPs[rir] = ips/total*100
    
RirIPs['US'] = US/total*100
RirIPs['CN'] = CN/total*100
RirIPs['UE'] = UE/total*100
    
print(RirIPs)    



{'LACNIC': 6.003890602396232, 'APNIC': 26.857541108042998, 'ARIN': 37.56584987985316, 'CN': 10.516117026681766, 'unknown': 1.5177573791974672, 'RIPE NCC': 24.684719658870705, 'US': 35.67664589464838, 'UE': 18.860250584811062, 'AFRINIC': 3.3702413716394353}
